In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import datasets, transforms
from torch.nn.functional import affine_grid, grid_sample, conv2d
from sim_moving import *

In [2]:
from crop import *
crop = Crop(digit_size=28, frame_size=64, CUDA=False, device=None)

In [5]:
num_videoes = 1000
T = 10
num_digits = 1
step_length = 0.2
frame_size = 64
PATH = '/home/hao/Research/amortized/Moving-MNIST/data/'
mnist_path = '/home/hao/Research/apg_data/mnist/'
simulator = BouncingMNIST(mnist_path=mnist_path, path=PATH, timesteps=T, num_digits=num_digits, step_length=step_length)

In [6]:
simulator.save_data(num_videoes=num_videoes)

In [ ]:
B = 3
S = 2
# T = 10
z_where = []
for i in range(B*S):
    X, _ = simulator.sim_trajectory()
    z_where.append(X.unsqueeze(0))
z_where = torch.cat(z_where, 0).view(S, B, 10, 2)

mnist = simulator.load_mnist()
digit = torch.from_numpy(mnist[150:150+S*B]).float().view(S, B, 28, 28)
frames = crop.digit_to_frame(digit, z_where)

videoes = frames.view(S*B, T, 64, 64)
num_cols = T
num_rows = S*B
fs = 15
gs = gridspec.GridSpec(num_rows, num_cols)
gs.update(left=0.0 , bottom=0.0, right=1.0, top=1.0, wspace=0.05, hspace=0.05)
fig = plt.figure(figsize=(fs, fs * num_rows / num_cols))
for i in range(num_rows):
    for j in range(num_cols):
        ax = fig.add_subplot(gs[i, j])
        ax.set_xticks([])
        ax.set_yticks([])
        ax.imshow(videoes[i, j, :, :], cmap='gray', vmin=0.0, vmax=1.0)

# digit_re = crop.frame_to_digit(frames, z_where)
# num_cols = T
# num_rows = 1*B
# fs = 15
# gs = gridspec.GridSpec(num_rows, num_cols)
# gs.update(left=0.0 , bottom=0.0, right=1.0, top=1.0, wspace=0.05, hspace=0.05)
# fig = plt.figure(figsize=(fs, fs * num_rows / num_cols))
# for i in range(num_rows):
#     for j in range(num_cols):
#         ax = fig.add_subplot(gs[i, j])
#         ax.set_xticks([])
#         ax.set_yticks([])
#         ax.imshow(digit_re[i, j, :, :], cmap='gray', vmin=0.0, vmax=1.0)

In [ ]:
digit_re = crop.frame_to_digit(frames[0], z_where)

In [ ]:
digit_re.shape

In [ ]:
a = simulator.load_mnist()[325]
plt.imshow(a, cmap='gray')

In [ ]:
# digit to frame
theta = torch.Tensor([[64/28, 0, -1*(64/28)*(36/64)], [0, 64/28, 1*(64/28)*(36/64)]])
grid = affine_grid(theta.unsqueeze(0), torch.Size((1, 1, 64, 64)))
output = grid_sample(torch.from_numpy(a).float().unsqueeze(0).unsqueeze(0), grid, mode='nearest')[0,0]
plt.imshow(output.data.numpy(), cmap='gray')
plt.show()
# # frame to digit
theta2 = torch.Tensor([[(28/64), 0, 1*(36/64)], [0, (28/64), -1*(36/64)]])
grid2 = affine_grid(theta2.unsqueeze(0), torch.Size((1, 1, 28, 28)))
output2 = grid_sample(output.unsqueeze(0).unsqueeze(0), grid2, mode='nearest')[0,0]
plt.imshow(output2.data.numpy(), cmap='gray')
plt.show()
## analytical solution
x = 1.
y = 0.0
top = int(y * 36)
left = int(x * 36)
canvas = np.zeros((64, 64))
canvas[top:top+28, left:left+28] = a
plt.imshow(canvas, cmap='gray')
plt.show()